---
### Universidad de Costa Rica
#### IE0405 - Modelos Probabilísticos de Señales y Sistemas

Primer semestre del 2022

---

* Estudiante: **Kenny Wu Wen**
* Carné: **C08592**
* Grupo: **1/2**

---
# `P5` - *Cadenas de Markov*

> Como un caso especial de los procesos aleatorios donde existen "estados" y "transiciones" entre estos estados, las cadenas de Markov modelan las probabilidades de transición y el tiempo de permanencia entre cada estado. En particular, la teoría de colas describe la evolución del número de elementos en la fila, dado un flujo de entrada y un flujo de salida.

---

## 1. - Procesos de nacimiento y muerte

Los procesos de nacimiento y muerte son un caso especial de las cadenas de Markov donde las transiciones solamente pueden ocurrir de un estado $i$ a un estado $i + 1$ ("nacimiento" o llegada) o $i - 1$ ("muerte" o salida). 

- Las llegadas están modeladas con un parámetro $\lambda$ de un proceso de Poisson, también llamado "intensidad".
- Las salidas, o tiempo de servicio, están modeladas exponencialmente con un parámetro $\nu$.

Dos componentes de este sistema son aleatorios:

- El **tiempo** que tarda "la máquina" (el sistema) en el estado $i$, que está modelado exponencialmente con un parámetro $\Omega_i$.
- La **transición** que hará de ese estado, que puede ser "hacia arriba" ($i+1$) con probabilidad $p_i$ o "hacia abajo" ($i-1$) con probabilidad $q_i$.

Generalmente es necesario encontrar la expresión para $\Omega_i$, $p_i$ y $q_i$ en términos de $\lambda$ y $\nu$ para conocer mejor la dinámica del sistema.

---
## 2. - Asignaciones del proyecto

### 2.1. - Problema de la gasolinera

> Supóngase que hay una gasolinera con tres entradas, los autos que entran arriban en forman de una corriente de Poisson con parámetro $\lambda$. La entrada norte tiene tasa de llegada de dos autos por minuto, la entrada este tiene una intensidad de dos autos por minuto y la entrada sur tiene un parámetro de un auto por minuto. El tiempo de servicio de cada dispensador de gasolina está exponencialmente distribuido con parámetro $\nu$ autos por minuto. El estado $X_t$ en el tiempo $t$ es la longitud de la cola. Inicialmente, en la gasolinera se enuentran tres dispensadores de gasolina funcionando pero, si la fila sobrepasa los cinco automóviles, abren otros dos dispensadores más. 

Suponga que el parámetro $\nu$ es la mediana entre los últimos dígitos del número de carné universitario de quienes realizan este proyecto. Ejemplo: A1384**3**, B8364**8** y C2317**7** tienen una mediana de 7.

- (30%) Encuentre los valores $p_i$, $q_i$ y $\Omega_i$ del espacio de estados anteriormente descrito.

### 2.2. - Simulaciones

- (30%) Diseñe una simulación tal que encuentre la duración promedio del estado $i$ y compárelo con el valor $1/\Omega_i$ calculado anteriormente, tanto para un número $i$ menor a cinco, como para un número $i$ mayor a cinco.
- (40%) Diseñe una simulación tal que muestre, por medio de una gráfica, la longitud de la cola a lo largo de 30 minutos.

#### Entrega de resultados

Crear un *notebook* nuevo de Jupyter y exportar como HTML.

---
**Universidad de Costa Rica** | Facultad de Ingeniería | Escuela de Ingeniería Eléctrica

&copy; 2022

---